In [3]:
# ======================  CONFIG  ======================
CFG = dict(
    csv_path   = r"",
    date_col   = "date",      
    lookback   = 120,         
    tau        = 20,          
    pred_len   = 96,          
    d_model    = 64,
    nhead      = 4,
    num_layers = 2,
    dropout    = 0.05,
    mlp_hidden = 80,
    lr         = 0.01,
    epochs     = 180,
    batch_size = 64,
    lamb_phys  = 1,
    lambs      = (1,1,1,1,1),  
    device     = "cuda" if __import__("torch").cuda.is_available() else "cpu",
    plot_each  = True
)
# =======================================================
import warnings, numpy as np, pandas as pd, torch, \
       torch.nn as nn, torch.nn.functional as F, matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from tqdm import trange, tqdm


def load_csv(path, date_col):
    df = pd.read_csv(path)
    if date_col and date_col in df.columns:
        df = df.drop(columns=[date_col])
    return df.astype(float)

def window_stack(a, L):
    return np.stack([a[i:i+L] for i in range(len(a)-L)], 0)

def causal_concepts(win, tau):
    y_prev, y_prev2 = win[:, -1], win[:, -2]
    c1 = win[:, -tau-1:-1].mean(-1)
    c2 = y_prev - y_prev2
    c3 = y_prev * c2
    seg = win[:, -tau-1:-1] - c1[:, None]
    fft = np.fft.rfft(seg, axis=-1)
    a1 = 2 * fft[:, 1].real / tau
    b1 = -2 * fft[:, 1].imag / tau
    c4 = 2 * np.sqrt(a1**2 + b1**2)
    c5 = seg.std(-1)
    return np.stack([c1, c2, c3, c4, c5], -1).astype(np.float32)



class Encoder(nn.Module):
    def __init__(self, L, d, h, nl, drop):
        super().__init__()
        self.emb = nn.Linear(1, d)
        layer = nn.TransformerEncoderLayer(d, h, d*4, drop, batch_first=True)
        self.trans = nn.TransformerEncoder(layer, nl)
        self.pos = nn.Parameter(torch.randn(L, d))
    def forward(self, x):
        h = self.emb(x.unsqueeze(-1)) + self.pos
        return self.trans(h)[:, -1]

class ConceptLayer(nn.Module):
    def __init__(self, d, hid=32, k=5):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(d, hid), nn.GELU(),
                                 nn.Linear(hid, k))
    def forward(self, h): return self.net(h)

class PINNHead(nn.Module):
    def __init__(self, k=5, hid=32):
        super().__init__()
        self.mlp = nn.Sequential(nn.Linear(k, hid), nn.GELU(),
                                 nn.Linear(hid, 1))
        self.beta0 = nn.Parameter(torch.zeros(1))
        self.betas = nn.Parameter(torch.zeros(k))
        self.gamma = nn.Parameter(torch.ones(1))
    def forward(self, C): return self.mlp(C).squeeze(-1)


def physics_residual(y_prev, C, head, lamb_phys, lambs):
    eps = 1e-6
    R1 = grad_or_zero(C[:,0].sum(), y_prev) - C[:,1]
    R2 = grad_or_zero(C[:,1].sum(), y_prev) - C[:,2]/(y_prev+eps)
    R3 = C[:,2] - y_prev*C[:,1]
    R4 = grad_or_zero((C[:,4]**2).sum(), y_prev) - 2*(y_prev-C[:,0])*C[:,1]

    rhs = head.beta0 + (C*head.betas).sum(-1) - head.gamma*(y_prev-C[:,0])
    Ry  = -rhs                                   # 离散近似

    w1,w2,w3,w4,wy = lambs
    phys = (w1*R1.pow(2)+w2*R2.pow(2)+w3*R3.pow(2)+w4*R4.pow(2)+wy*Ry.pow(2)).mean()
    return lamb_phys*phys

def train_series(arr_std):
    dev=CFG['device']; L=CFG['lookback']
    ds = SeriesDataset(arr_std,L,CFG['tau'])
    dl = DataLoader(ds,batch_size=CFG['batch_size'],shuffle=True,drop_last=True)


# ---------- 推理 ----------
def predict_last96(arr_std,enc,con,head,scaler):
    dev=CFG['device']; L=CFG['lookback']; P=CFG['pred_len']
    wins=window_stack(arr_std,L)[-P:]
    with torch.no_grad():
        
          y_true=scaler.inverse_transform(arr_std[-P:].reshape(-1,1)).ravel()
    return y_true,y_hat

def main():
    df=load_csv(CFG['csv_path'],CFG['date_col'])
    ms,ma=[],[]
    for col in tqdm(df.columns,desc='Column Loop'):
        s=df[col].values.astype(float)
        scaler=StandardScaler().fit(s[:-CFG['pred_len']].reshape(-1,1))
        s_std=scaler.transform(s.reshape(-1,1)).ravel()
        try:
            enc,con,head=train_series(s_std)
            y_true,y_pred=predict_last96(s_std,enc,con,head,scaler)
        except RuntimeError as e:
            warnings.warn(f"{col} fail: {e}"); continue
        ms.append(((y_pred-y_true)**2).mean())
        ma.append(np.abs(y_pred-y_true).mean())
        if CFG['plot_each']:
            plt.figure(figsize=(8,3))
            plt.plot(y_true,label='True'); plt.plot(y_pred,label='Pred')
            plt.title(f'{col}  –  Last {CFG["pred_len"]} steps')
            plt.legend(); plt.tight_layout(); plt.show()
    if ms:
        print(f'\nmean MSE: {np.mean(ms):.4f}   mean MAE: {np.mean(ma):.4f}')
    else:
        print("fail。")

if __name__=="__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: ''